In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import cv2
import glob
import tensorflow as tf
import tensorflow.contrib.eager as tfe

tfe.enable_eager_execution()


In [ ]:
images=[]
for file in glob.glob("./Train/CameraRGB/*.png"):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    images.append(img)

masks = []
for file in glob.glob("./Train/CameraSeg/*.png"):
    img = cv2.imread(file)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    masks.append(img)

In [ ]:
def multi_plot(images):
    plt.figure(figsize=(20,20))
    count = len(images)
    
    for i in range(count):
        plt.subplot(1, count, i+1)
        plt.imshow(images[i])

multi_plot([images[0], images[1], images[2], images[3][:, :, 1]])

In [ ]:
def preprocess_labels(label_image):
    road = np.zeros_like(label_image[:, :, 0])
    lane_marking_pixels = (label_image[:,:,0] == 6).nonzero()
    road_pixels = (label_image[:,:,0] == 7).nonzero()
    road[lane_marking_pixels] = 1
    road[road_pixels] = 1

    car = np.zeros_like(road)
    vehicle_pixels = (label_image[:,:,0] == 10).nonzero()
    # Isolate vehicle pixels associated with the hood (y-position > 496)
    hood_indices = (vehicle_pixels[0] >= 496).nonzero()[0]
    hood_pixels = (vehicle_pixels[0][hood_indices], \
                   vehicle_pixels[1][hood_indices])
    car[vehicle_pixels] = 1
    car[hood_pixels] = 0
    return np.stack([road, car], axis=2)

# new_label = preprocess_labels(masks[1])
# print(new_label[:, :, 0].shape)
# plt.imshow(new_label[:, :, 1])

In [ ]:
processed_masks = [preprocess_labels(mask) for mask in masks]

In [ ]:
for index in range(15, 20):
    multi_plot([images[index], processed_masks[index][:, :, 0], processed_masks[index][:, :, 1]])

In [ ]:
class DownSample(tf.keras.Model):
    def __init__(self, filters):
        super(DownSample, self).__init__()
        self.conv = tf.keras.layers.Conv2D(filters, kernel_size=(3, 3), padding='same', strides=(2, 2))
        self.prelu = tf.keras.layers.PReLU()
        self.maxpool = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding='same')
        self.concat = tf.keras.layers.Concatenate()
        
    def call(self, inputs):
        pool = self.maxpool(inputs)
        x = self.conv(inputs)
        x = self.prelu(x)
        x = self.concat([x, pool])
        return x
    
model = DownSample(filters=13)
batch = tf.random_normal((1, 600, 800, 3))
%time out = model(batch)

print(out.shape)


In [ ]:
class DilationLayer(tf.keras.Model):
    def __init__(self, filters):
        super(DilationLayer, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size=(1, 1), padding='same', dilation_rate=(1,1))
        self.prelu1 = tf.keras.layers.PReLU()
        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size=(3, 3), padding='same', dilation_rate=(2,2))
        self.prelu2 = tf.keras.layers.PReLU()
        self.batchnorm = tf.keras.layers.BatchNormalization()
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.prelu1(x)
        x = self.conv2(x)
        x = self.prelu2(x)
        x = self.batchnorm(x)
        return x
    
model = DilationLayer(filters=13)
batch = tf.random_normal((1, 75, 100, 25))
%time out = model(batch)

print(out.shape)


In [ ]:
class UpSample(tf.keras.Model):
    def __init__(self, filters):
        super(UpSample, self).__init__()
        self.conv1 = tf.keras.layers.Conv2DTranspose(filters, kernel_size=(3, 3), strides=(2,2), padding='same')
        self.prelu1 = tf.keras.layers.PReLU()
        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size=(3, 3), padding='same', dilation_rate=(1,1))
        self.prelu2 = tf.keras.layers.PReLU()
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.prelu1(x)
        x = self.conv2(x)
        x = self.prelu2(x)
        return x
    
model = UpSample(filters=13)
batch = tf.random_normal((1, 75, 100, 25))
%time out = model(batch)

print(out.shape)


In [ ]:
class DilatedCNN(tf.keras.Model):
    def __init__(self):
        super(DilatedCNN, self).__init__()
        self.down1 = DownSample(filters=13)
        self.down2 = DownSample(filters=24)
        self.down3 = DownSample(filters=36)
        
        self.dilation1 = DilationLayer(filters=76)
        self.dilation2 = DilationLayer(filters=76)
        self.dilation3 = DilationLayer(filters=76)
        
        self.up1 = UpSample(filters=36)
        self.up1_concat = tf.keras.layers.Concatenate()
        
        self.up2 = UpSample(filters=18)
        self.up2_concat = tf.keras.layers.Concatenate()
        
        self.last_concat = tf.keras.layers.Concatenate()
        self.last_conv1 = tf.keras.layers.Conv2DTranspose(9, kernel_size=(3, 3), strides=(2,2), padding='same')
        self.last_prelu = tf.keras.layers.PReLU()
        self.last_layer = tf.keras.layers.Conv2D(2, kernel_size=(3, 3), 
                                                 padding='same', dilation_rate=(1,1), activation=tf.nn.sigmoid)
#         self.up3 = UpSample(filters=2)
        
        self.beta_road = tf.constant(0.5)
        self.beta_car = tf.constant(2.)

    def call(self, inputs):
        d1 = self.down1(inputs)
        d2 = self.down2(d1)
        x = self.down3(d2)
        x = self.dilation1(x)
        x = self.dilation2(x)
        x = self.dilation3(x)
        x = self.up1(x)
        x = self.up1_concat([x, d2])
        x = self.up2(x)
        x = self.up2_concat([x, d1])
        x = self.last_conv1(x)
        x = self.last_concat([x, inputs])
        x = self.last_layer(x)
        return x
        
    def loss(self, predictions, targets):
        """road = layer [0], car = layer [1]"""
        
        
        y = self(inputs)
        loss = tf.losses.softmax_cross_entropy(onehot_labels=targets, logits=y)
        return loss

    def loss_dice_coef(self, y_hat, y):
#         y = tf.reshape(y, [-1])
#         y_hat = tf.reshape(y_hat, [-1])
        
        intersection = tf.reduce_sum(y * y_hat)
        top = 2 * intersection + 1
        bottom = tf.reduce_sum(y) + tf.reduce_sum(y_hat) + 1
        return -top/bottom
#         return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_hat)
    
    def grad(self, x, y):
        with tfe.GradientTape() as tape:
            y_hat = self(x)
            loss_value = self.loss_dice_coef(y_hat, y)
        return tape.gradient(loss_value, self.variables)
    
    def train(self, x, y, optimizer):
        grads = self.grad(x, y)
        optimizer.apply_gradients(zip(grads, self.variables),
                                  global_step=tf.train.get_or_create_global_step())
    
    def save(self):
        self.save_weights("model_weights.h5")

    def load(self):
        self.load_weights("model_weights.h5")
    
model = DilatedCNN()
x = tf.convert_to_tensor(images[0:1], dtype=tf.float32)
print(x.shape)

%time test = model(x)
print(test.shape)
y_hat = np.array([[1, 0, 1, 1],
                  [1, 0, 1, 1],
                  [1, 1, 0, 1],
                  [1, 1, 0, 1]
                 ])

y = np.array(    [[1, 0, 1, 0],
                  [1, 0, 1, 0],
                  [1, 1, 0, 1],
                  [1, 1, 0, 1]
                 ])

out = model.loss_dice_coef(y_hat, y)
print(out)

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)

In [ ]:
model.load_weights("model_weights.h5")
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

In [ ]:
# image = images[2]
# mask = processed_masks[2]
# multi_plot([image, mask[:, :, 0], mask[:, :, 1]])

x = tf.constant(images[: 5], dtype=tf.float32)
y = tf.constant(processed_masks[:5], dtype=tf.float32)

for i in range(200):
    model.train(x, y, optimizer)
    if i % 10 == 0:
        y_hat = model(x)
        print(i, model.loss_dice_coef(y_hat, y))

print("done")
%time y_hat = model(x)
index = 2
multi_plot([images[index], y_hat[index][:, :, 0], y_hat[index][:, :, 1]])

In [ ]:
%time y_hat = model(x[0:1])

In [ ]:
model.save_weights("model_weights.h5")